In [1]:
from preamble import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [3]:
df = pd.read_csv('./data/j_before_feature_importance.csv')
data = pd.read_csv('./data/j_before_feature_importance_with_datetime.csv')  # datetime 정보

In [4]:
df.dropna(inplace=True)

In [5]:
df.head()

,temperature,precipitation,wind_speed,humidity,atmospheric_pressure,snow,outflow,input,dayofmonth,dayofweek,quarter,month,year,dayofyear,week,hour,minute,second,is_weekend,is_holiday,lag_1h,lag_2h,lag_3h,lag_24h,lag_168h,rolling_3h_avg,rolling_6h_max,rolling_6h_avg,rolling_24h_max,change_rate_1h,rolling_7d_std,delta_24h,daily_range_lag_1d
168,-0.9,0.0,1.7,78.0,1016.8,0.0,8357.48,6869.42,8,6,1,1,2023,8,1,0,0,0,1,1,8426.26,8376.72,16549.00,8655.74,4672.98,8386.82,16549.00,11937.76,16549.0,-8.16e-03,2186.71,-0.03,8803.18
169,-1.0,0.0,0.9,78.0,1016.9,0.0,8628.98,7647.39,8,6,1,1,2023,8,1,1,0,0,1,1,8357.48,8426.26,8376.72,8966.82,6425.49,8470.91,16549.00,11147.91,16549.0,3.25e-02,2175.26,-0.04,7377.15
170,-1.5,0.0,1.0,80.0,1017.4,0.0,6164.87,4265.54,8,6,1,1,2023,8,1,2,0,0,1,1,8628.98,8357.48,8426.26,8643.03,8620.11,7717.11,16549.00,9417.22,16549.0,-2.86e-01,2188.92,-0.29,7377.15
171,-1.7,0.0,1.6,81.0,1018.3,0.0,5610.79,4194.09,8,6,1,1,2023,8,1,3,0,0,1,1,6164.87,8628.98,8357.48,8104.00,8697.04,6801.55,8628.98,7594.18,16549.0,-8.99e-02,2207.83,-0.31,7377.15
172,-1.9,0.0,1.5,81.0,1018.2,0.0,5614.06,4245.52,8,6,1,1,2023,8,1,4,0,0,1,1,5610.79,6164.87,8628.98,7819.86,9543.70,5796.57,8628.98,7133.74,16549.0,5.84e-04,2226.98,-0.28,7377.15


In [8]:
X, y = df.reset_index(drop=True), df['outflow'].values.reshape(-1, 1)

In [10]:
X.head()

,temperature,precipitation,wind_speed,humidity,atmospheric_pressure,snow,outflow,input,dayofmonth,dayofweek,quarter,month,year,dayofyear,week,hour,minute,second,is_weekend,is_holiday,lag_1h,lag_2h,lag_3h,lag_24h,lag_168h,rolling_3h_avg,rolling_6h_max,rolling_6h_avg,rolling_24h_max,change_rate_1h,rolling_7d_std,delta_24h,daily_range_lag_1d
0,-0.9,0.0,1.7,78.0,1016.8,0.0,8357.48,6869.42,8,6,1,1,2023,8,1,0,0,0,1,1,8426.26,8376.72,16549.00,8655.74,4672.98,8386.82,16549.00,11937.76,16549.0,-8.16e-03,2186.71,-0.03,8803.18
1,-1.0,0.0,0.9,78.0,1016.9,0.0,8628.98,7647.39,8,6,1,1,2023,8,1,1,0,0,1,1,8357.48,8426.26,8376.72,8966.82,6425.49,8470.91,16549.00,11147.91,16549.0,3.25e-02,2175.26,-0.04,7377.15
2,-1.5,0.0,1.0,80.0,1017.4,0.0,6164.87,4265.54,8,6,1,1,2023,8,1,2,0,0,1,1,8628.98,8357.48,8426.26,8643.03,8620.11,7717.11,16549.00,9417.22,16549.0,-2.86e-01,2188.92,-0.29,7377.15
3,-1.7,0.0,1.6,81.0,1018.3,0.0,5610.79,4194.09,8,6,1,1,2023,8,1,3,0,0,1,1,6164.87,8628.98,8357.48,8104.00,8697.04,6801.55,8628.98,7594.18,16549.0,-8.99e-02,2207.83,-0.31,7377.15
4,-1.9,0.0,1.5,81.0,1018.2,0.0,5614.06,4245.52,8,6,1,1,2023,8,1,4,0,0,1,1,5610.79,6164.87,8628.98,7819.86,9543.70,5796.57,8628.98,7133.74,16549.0,5.84e-04,2226.98,-0.28,7377.15


In [11]:
train_size = int(len(X) * 0.7)

trainset_feature = X[:train_size]
trainset_target = y[:train_size]
testset_feature = X[train_size:]
testset_target = y[train_size:]

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler_feature = MinMaxScaler()
train_feature_scaled = scaler_feature.fit_transform(trainset_feature)
test_feature_scaled = scaler_feature.transform(testset_feature)

scaler_target = MinMaxScaler()
train_target_scaled = scaler_target.fit_transform(trainset_target)
test_target_scaled = scaler_target.transform(testset_target)

In [13]:
valid_size = int(train_feature_scaled.shape[0] * 0.9)

train_feature_scale = train_feature_scaled[:valid_size]
train_target_scale = train_target_scaled[:valid_size]
valid_feature_scaled = train_feature_scaled[valid_size:]
valid_target_scaled = train_target_scaled[valid_size:]

In [15]:
import pickle

with open('scaler_feature_lstm_v10.pkl', 'wb') as f:
  pickle.dump(scaler_feature, f)
with open('scaler_target_lstm_v10.pkl', 'wb') as f:
  pickle.dump(scaler_target, f)

In [ ]:
def split_dataset(data, target, seq_len, pred_len):
  X, y = [], []
  for i in range(data.shape[0] - (seq_len + pred_len) + 1):
    X.append(data[i:i + seq_len, :])
    y.append(target[i + seq_len:i + seq_len + pred_len])
  return np.array(X), np.array(y)

In [18]:
seq_len = 168  # 과거 7일 (시간 단위)
pred_len = 1  # 이후 1시간 예측

X_train, y_train = split_dataset(train_feature_scaled, train_target_scaled, seq_len, pred_len)
X_valid, y_valid = split_dataset(valid_feature_scaled, valid_target_scaled, seq_len, pred_len)
X_test, y_test = split_dataset(test_feature_scaled, test_target_scaled, seq_len, pred_len)

print(X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape)

(10580, 168, 33) (907, 168, 33) (4439, 168, 33) (10580, 1, 1) (907, 1, 1) (4439, 1, 1)


In [19]:
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
  def __init__(self, X, y):
    self.data = torch.tensor(X, dtype=torch.float32)
    self.labels = torch.tensor(y, dtype=torch.float32)

  def __len__(self):
    return len(self.data)  # 데이터의 길이를 반환

  def __getitem__(self, idx):
    return self.data[idx], self.labels[idx] # 데이터와 레이블을 함께 반환

In [20]:
from torch.utils.data import DataLoader

batch_size = 128
train_dataset = SequenceDataset(X_train, y_train)
valid_dataset = SequenceDataset(X_valid, y_valid)
test_dataset = SequenceDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

In [21]:
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(LSTMModel, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    lstm_out, _ = self.lstm(x)
    last_out = lstm_out[:, -1, :]
    out = self.fc(last_out)
    return out

input_size = X_train.shape[2]
hidden_size = 50
output_size = TARGET_LENGTH

model = LSTMModel(input_size, hidden_size, output_size).to(device)
print(model)

LSTMModel(
  (lstm): LSTM(33, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)


In [22]:
import torch.optim as optim

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
def evaluate(model, valid_dataset, loss_fn, device):
  val_loss = 0
  model.eval()
  with torch.no_grad():
    for X, y in valid_dataset:
      X_batch, y_batch = X.to(device), y.to(device)
      out = model(X_batch)
      loss = loss_fn(out, y_batch)
      val_loss += loss.item()
  return val_loss

In [24]:
num_epochs = 300
best_valid_loss = float('inf')
loss_history = []
model.to(device)

for epoch in range(num_epochs):
  model.train() 
  train_loss = 0
  for X, y in train_loader:
    X_batch, y_batch = X.to(device), y.to(device)

    out = model(X_batch)
    loss = loss_fn(out, y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

  loss_history.append(train_loss / len(train_loader.dataset))
  valid_loss = evaluate(model, valid_loader, loss_fn, device)
  print(f'Epoch : {epoch + 1} / {num_epochs}')
  print(f'Train Loss : {train_loss}')
  print(f'Valid Loss : {valid_loss}')
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), './model/1hour_best_lstm_model.pt')

c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([128, 1, 1])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([84, 1, 1])) that is different to the input size (torch.Size([84, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\DevTool\anaconda3\envs\torch_book\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([11, 1, 1])) that is different to the input size (torch.Size([11, 1])). This will likely lead to incorrect results due to broadcasting. 

Epoch : 1 / 300
Train Loss : 2.6842675302177668
Valid Loss : 0.14814091101288795
Epoch : 2 / 300
Train Loss : 1.6871117819100618
Valid Loss : 0.14710033684968948
Epoch : 3 / 300
Train Loss : 1.646340380422771
Valid Loss : 0.14670425094664097
Epoch : 4 / 300
Train Loss : 1.6364772012457252
Valid Loss : 0.14514208398759365
Epoch : 5 / 300
Train Loss : 1.6318302396684885
Valid Loss : 0.14546384289860725
Epoch : 6 / 300
Train Loss : 1.6291987746953964
Valid Loss : 0.14536708500236273
Epoch : 7 / 300
Train Loss : 1.6488486165180802
Valid Loss : 0.14427818823605776
Epoch : 8 / 300
Train Loss : 1.632421476766467
Valid Loss : 0.14683786127716303
Epoch : 9 / 300
Train Loss : 1.6400932352989912
Valid Loss : 0.14426002837717533
Epoch : 10 / 300
Train Loss : 1.6364300902932882
Valid Loss : 0.1442729691043496
Epoch : 11 / 300
Train Loss : 1.6414349023252726
Valid Loss : 0.14409924484789371
Epoch : 12 / 300
Train Loss : 1.6290398798882961
Valid Loss : 0.14475528243929148
Epoch : 13 / 300
Train Loss 

In [25]:
model.load_state_dict(torch.load('./model/1hour_best_lstm_model.pt'))
model.to(device)

C:\Users\user\AppData\Local\Temp\ipykernel_13592\3127988678.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model/1hour_best_lstm_mod

LSTMModel(
  (lstm): LSTM(33, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)

In [26]:
model.eval()

y_pred_list = []
with torch.no_grad():
  for X_batch, _ in test_loader:
    X_batch = X_batch.to(device)
    y_pred = model(X_batch).cpu().numpy()
    y_pred_list.extend(y_pred)

y_pred = np.array(y_pred_list)

In [27]:
print(len(train_dataset))
print(len(valid_dataset))

10580
907


In [28]:
print(f"y_test 크기: {len(y_test)}")
print(f"y_pred 크기: {len(y_pred)}")

y_test 크기: 4439
y_pred 크기: 4439


In [29]:
from sklearn.metrics import mean_squared_error, r2_score
def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Test RMSE: {rmse:.4f}')
print(f'Test MAPE: {mape:.4f}%')
print(f'Test R-squared: {r2:.4f}')

ValueError: Found array with dim 3. None expected <= 2.

In [ ]:
plt.figure(figsize=(20, 8))

start_index = train_size + SEQ_LENGTH
end_index = start_index + len(y_test)

datetime_range = data['datetime'][start_index:end_index].values
if len(datetime_range) < len(y_pred):
  print("Warning: Datetime range is shorter than prediction length. Adjusting range.")
  y_pred = y_pred[:len(datetime_range)]

y_test_plot = y_test[:len(datetime_range)]

plt.plot(datetime_range, y_test_plot, label='Actual')
plt.plot(datetime_range, y_pred, label='Predicted')

plt.xticks([])
plt.xlabel('Datetime')
plt.ylabel('Outflow')
plt.title('Actual vs Predicted Outflow (LSTM)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def plot_predictions_weekly(y_true, y_pred, datetime_data, start_index, plot_title):
  plt.figure(figsize=(20, 6))

  datetime_range = datetime_data[start_index:start_index + 168].values  # 일주일 (168시간)

  y_true_plot = y_true[:168]
  y_pred_plot = y_pred[:168]

  plt.plot(datetime_range, y_true_plot, label='Actual')
  plt.plot(datetime_range, y_pred_plot, label='Predicted')

  plt.xlabel('Datetime')
  plt.ylabel('Outflow')
  plt.title(plot_title)
  plt.legend()
  plt.xticks([])
  plt.tight_layout()
  plt.show()

start_index_start = train_size + SEQ_LENGTH
plot_predictions_weekly(y_test, y_pred, data['datetime'], start_index_start, 'First Week Predictions (LSTM)')

start_index_end = train_size + SEQ_LENGTH + len(y_test) - 168
plot_predictions_weekly(y_test[-168:], y_pred[-168:], data['datetime'], len(data) - 168, 'Last Week Predictions (LSTM)')